In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [10]:
class HairCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

model = HairCNN().to(device)

total_params = sum(p.numel() for p in model.parameters())
total_params

20073473

In [11]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

criterion, optimizer

(BCEWithLogitsLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.002
     maximize: False
     momentum: 0.8
     nesterov: False
     weight_decay: 0
 ))

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (preds == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6654, Acc: 0.6262, Val Loss: 0.6102, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5479, Acc: 0.7087, Val Loss: 0.6303, Val Acc: 0.6468
Epoch 3/10, Loss: 0.4858, Acc: 0.7638, Val Loss: 0.6991, Val Acc: 0.5970
Epoch 4/10, Loss: 0.4806, Acc: 0.7650, Val Loss: 0.6097, Val Acc: 0.7015
Epoch 5/10, Loss: 0.4403, Acc: 0.7863, Val Loss: 0.6215, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3493, Acc: 0.8500, Val Loss: 0.6453, Val Acc: 0.6716
Epoch 7/10, Loss: 0.3005, Acc: 0.8775, Val Loss: 0.6953, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2746, Acc: 0.8938, Val Loss: 0.6370, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2119, Acc: 0.9225, Val Loss: 0.6828, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1419, Acc: 0.9525, Val Loss: 0.8263, Val Acc: 0.6965


In [13]:
acc_arr = np.array(history['acc'])
loss_arr = np.array(history['loss'])

median_train_acc = np.median(acc_arr)
std_train_loss = np.std(loss_arr)

print("Train accuracies:", acc_arr)
print("Train losses:", loss_arr)
print(f"Median train accuracy (Q3): {median_train_acc:.4f}")
print(f"Std of train loss (Q4): {std_train_loss:.4f}")

Train accuracies: [0.62625 0.70875 0.76375 0.765   0.78625 0.85    0.8775  0.89375 0.9225
 0.9525 ]
Train losses: [0.66539761 0.5478831  0.48576985 0.4805766  0.44032146 0.34926566
 0.30053565 0.27462086 0.21189461 0.14193405]
Median train accuracy (Q3): 0.8181
Std of train loss (Q4): 0.1539


In [14]:
augmented_train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = datasets.ImageFolder(train_dir, transform=augmented_train_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=batch_size, shuffle=True)

num_epochs_aug = 10
history_aug = {'val_loss': [], 'val_acc': []}

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (preds == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"[AUG] Epoch {epoch+1}/{num_epochs_aug}, "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

[AUG] Epoch 1/10, Train Loss: 0.6602, Train Acc: 0.6613, Val Loss: 0.7088, Val Acc: 0.6866
[AUG] Epoch 2/10, Train Loss: 0.5538, Train Acc: 0.7150, Val Loss: 0.6008, Val Acc: 0.7065
[AUG] Epoch 3/10, Train Loss: 0.5338, Train Acc: 0.7188, Val Loss: 0.5543, Val Acc: 0.7413
[AUG] Epoch 4/10, Train Loss: 0.5063, Train Acc: 0.7575, Val Loss: 0.5415, Val Acc: 0.7114
[AUG] Epoch 5/10, Train Loss: 0.4769, Train Acc: 0.7700, Val Loss: 0.5929, Val Acc: 0.7015
[AUG] Epoch 6/10, Train Loss: 0.4684, Train Acc: 0.7725, Val Loss: 0.5302, Val Acc: 0.7313
[AUG] Epoch 7/10, Train Loss: 0.4527, Train Acc: 0.7812, Val Loss: 0.4919, Val Acc: 0.7612
[AUG] Epoch 8/10, Train Loss: 0.4577, Train Acc: 0.7887, Val Loss: 0.5146, Val Acc: 0.7313
[AUG] Epoch 9/10, Train Loss: 0.4423, Train Acc: 0.7925, Val Loss: 0.5061, Val Acc: 0.7711
[AUG] Epoch 10/10, Train Loss: 0.4337, Train Acc: 0.7987, Val Loss: 0.4928, Val Acc: 0.7413


In [15]:
val_loss_aug = np.array(history_aug['val_loss'])
val_acc_aug = np.array(history_aug['val_acc'])

mean_test_loss_aug = np.mean(val_loss_aug)
avg_last5_test_acc = np.mean(val_acc_aug[-5:])

print("Val losses (aug):", val_loss_aug)
print("Val accuracies (aug):", val_acc_aug)
print(f"Mean test loss over 10 aug epochs (Q5): {mean_test_loss_aug:.4f}")
print(f"Average test acc over last 5 aug epochs (Q6): {avg_last5_test_acc:.4f}")

Val losses (aug): [0.70876624 0.6007967  0.55425786 0.54153171 0.59289752 0.53020695
 0.49186589 0.5145741  0.50614088 0.49276404]
Val accuracies (aug): [0.68656716 0.70646766 0.74129353 0.71144279 0.70149254 0.73134328
 0.76119403 0.73134328 0.77114428 0.74129353]
Mean test loss over 10 aug epochs (Q5): 0.5534
Average test acc over last 5 aug epochs (Q6): 0.7473
